# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.16.6 requires protobuf!=4.21.0,<5,>=3.19.0; sys_platform != "linux", but you have protobuf 5.27.2 which is incompatible.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.2 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> **Answer:**
>> Qdrant uses several techniques to make it performant
>> - Qdrant is written in Rust programming language, which makes it performant and reliable
>> - Qdrant uses Hierarchical navigable Small World (HNSW) indexing for fast retrieval
>> - Qdrant offers several implementations like memory, on-disk, on-cloud to provide best performance and cost optimization options
>> - Qdrant automatically enables indexes and memmap storage when a certain number of records is reached, optimizing performance as the database size grows
>> - Qdrant employs Vaccuum Optimizer, which combines multiple small segments into larger ones to optimize search performance
>> - Qdrant uses quantization for more efficient storage and faster retrieval 

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '036019aea40b49efa31034a9e04ec300', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

> **Answer:** The LCEL chain above is a series of steps piped from left and right, where each step takes input from the previous step and sends output to the next step. It starts with capturing user `question` and sending it to `retriever` which sets the `context`. The `context` is passed through to the LLM along with the `question` to the `ChatPrompt` which provides the `response`. The final output is the `response, context` tuple. The same is represented below as ASCII art. 

In [26]:
!pip install -qU grandalf

In [19]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

Let's test it out!

In [28]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of the industry.


In [29]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '85097ac9c48b4f3b802d6b22f00d86dc', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [30]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

> **Answer:** We want to split our documents using different parameters when creating synthetic data 
> - Purpose of synthetic data is different than the documents used for context setting. Synthetic needs to represent wide variety of scenarios that the model and RAG encounters than what it has been optimized for. So, the different parameters allows us to drive evaluation using different set of data
> - Retrieval data for context setting needs to be optimized for retrieval and locking in a single idea or concept inside one instance without filling up memory unnecessarily. This is different from the purpose of synthetic data. So, the parameters are different.

In [31]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#Reducing to first 500 rows to avoid OpenAI rate limits
eval_documents = eval_documents[0:500]
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> **Answer:** 
> 1. `simple` refers to basic retrieval and generation. It evaulating simple question capabilities. 
> 2. `reasoning` refers to more complex task that need reasoning beyond simple question, answering and solving multiple-step process and drawing inferences, not directly evident in the data.
> 3. `multi_context` refers to system ability to synthesize multiple sources or contexts of data. It signifies ability to draw data from other documents and domains to provide an answer.

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [34]:
testset.test_data[0]

DataRow(question='How does winning impact the retention of great people in companies?', contexts=['Part 2: Retaining great people\nThis post is about retaining great people, particularly at big com-\npanies in industries like technology, where stock options matter\nand where people can relatively easily move from one company\nto another.\nActually, I lied. This post isn’t really about retention at all.\nIt’s about winning.\nLet me explain:\nCompanies that are winning — even really big, old ones — never\nhave a retention problem. Everyone wants to stay, and when some-\none does leave, it’s really easy to get someone great to take her\nplace.\nCompanies that have a retention problem usually have a win-'], ground_truth="Companies that are winning never have a retention problem. Everyone wants to stay, and when someone does leave, it's really easy to get someone great to take her place.", evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/T

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [35]:
test_df = testset.to_pandas()

In [36]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does winning impact the retention of great...,[Part 2: Retaining great people\nThis post is ...,Companies that are winning never have a retent...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does the constant change in the world make...,[Part 1: Opportunity\nThe Hrst rule of career ...,The constant change in the world makes it diff...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,"What is the concept of ""Task Relevant Maturity...","[SpeciXcally, there are times and situations w...",The concept of 'Task Relevant Maturity' refers...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,Why is it important to focus on strength rathe...,[manage executives. It turns out that just abo...,"When managing executives, it is important to f...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What analogy is used to describe the interacti...,[pany’s permission to do something. Or maybe a...,The analogy used to describe the interaction b...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,Why do some entrepreneurs choose to hold back ...,[need it.\nWhat if you don’t want to raise tha...,Some entrepreneurs choose to hold back on rais...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,Why is it important to seek out and create opp...,[sented with an opportunity like one of the ab...,Both opportunities that present themselves and...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,Why is it important to be in a dynamic and exc...,[they might be.\nNever worry about being a sma...,Being in a dynamic and exciting pond is import...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What are some examples of situations where mis...,[— put yourself in situations where you will s...,missing a critical piece of information in a f...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does Nassim Nicholas Taleb describe the co...,"[do, just sit down and tease apart the risks —...",Many people do not realize they are getting a ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [37]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [38]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [47]:
from datasets import Dataset

# Truncating to avoid OpenAI Rate limits
TRUNC_SIZE = 10
response_dataset = Dataset.from_dict({
    "question" : test_questions[0:TRUNC_SIZE],
    "answer" : answers[0:TRUNC_SIZE],
    "contexts" : contexts[0:TRUNC_SIZE],
    "ground_truth" : test_groundtruths[0:TRUNC_SIZE]
})

Let's take a peek and see what that looks like!

In [48]:
response_dataset[0]

{'question': 'How does winning impact the retention of great people in companies?',
 'answer': 'Winning impacts the retention of great people in companies because great people want to work at a winner.',
 'contexts': ['The only way a company in that situation can retain great peo-\nple is to start winning again.\nGreat people want to work at a winner.\nAll the raises, perks, and HR-sponsored “company values” draa-',
  'and where people can relatively easily move from one company\nto another.\nActually, I lied. This post isn’t really about retention at all.\nIt’s about winning.\nLet me explain:',
  'one does leave, it’s really easy to get someone great to take her\nplace.\nCompanies that have a retention problem usually have a win-\nning problem. Or rather, a “not winning” problem.',
  'Part 2: Retaining great people\nThis post is about retaining great people, particularly at big com-\npanies in industries like technology, where stock options matter'],
 'ground_truth': "Companies that a

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [49]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [50]:
print(response_dataset.num_rows)

10


All that's left to do is call "evaluate" and away we go!

In [51]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 50/50 [01:25<00:00,  1.72s/it]


In [52]:
results

{'faithfulness': 0.8500, 'answer_relevancy': 0.9336, 'context_recall': 0.6667, 'context_precision': 0.7556, 'answer_correctness': 0.5340}

In [53]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How does winning impact the retention of great...,Winning impacts the retention of great people ...,[The only way a company in that situation can ...,Companies that are winning never have a retent...,1.0,0.951926,1.000000,0.805556,0.226177
1,How does the constant change in the world make...,The constant change in the world makes it diff...,[Trying to plan your career is an exercise in ...,The constant change in the world makes it diff...,1.0,0.943412,1.000000,1.000000,0.366915
2,"What is the concept of ""Task Relevant Maturity...","The concept of ""Task Relevant Maturity"" is des...","[book High Output Management, where he describ...",The concept of 'Task Relevant Maturity' refers...,1.0,0.933060,1.000000,0.916667,0.847602
3,Why is it important to focus on strength rathe...,It is important to focus on strength rather th...,[back — perhaps abstractly — about all of her ...,"When managing executives, it is important to f...",1.0,0.912358,0.333333,0.916667,0.406620
4,What analogy is used to describe the interacti...,"The analogy used is ""there is potentially a to...",[There are times in the life of a startup when...,The analogy used to describe the interaction b...,1.0,0.883689,0.000000,0.583333,0.206945
5,Why do some entrepreneurs choose to hold back ...,Some entrepreneurs choose to hold back on rais...,"[a higher valuation, and give away less of the...",Some entrepreneurs choose to hold back on rais...,1.0,0.859759,1.000000,0.916667,0.887176
6,Why is it important to seek out and create opp...,It is important to seek out and create opportu...,[One of the single best ways you can maximize ...,Both opportunities that present themselves and...,1.0,1.000000,1.000000,0.916667,0.732039
7,Why is it important to be in a dynamic and exc...,It is important to be in a dynamic and excitin...,[Third rule:\nIn a rapidly changing Held like ...,Being in a dynamic and exciting pond is import...,1.0,1.000000,1.000000,0.916667,0.997891
8,What are some examples of situations where mis...,"Launching a product and seeing it tank, being ...","[project, launching a product and seeing it ta...",missing a critical piece of information in a f...,0.5,0.853706,0.333333,0.000000,0.206630
9,How does Nassim Nicholas Taleb describe the co...,Nassim Nicholas Taleb describes the concept of...,[of a great opportunity. They oaen won’t.\nOne...,Many people do not realize they are getting a ...,0.0,0.997956,0.000000,0.583333,0.461882


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [54]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [55]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [56]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [57]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [58]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have information about any feud involving Taylor Swift.


In [59]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [60]:
print(response["answer"])

The text does not provide any information about a feud or conflict between individuals or groups. It mainly discusses factors that contribute to success or failure in business, human behavior, and decision-making processes.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [61]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [65]:
# Reduced size to fit OpenAI rate limits
REDUCED_SIZE = int(TRUNC_SIZE/2)
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions[0:REDUCED_SIZE],
    "answer" : answers[0:REDUCED_SIZE],
    "contexts" : contexts[0:REDUCED_SIZE],
    "ground_truth" : test_groundtruths[0:REDUCED_SIZE]
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [66]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 25/25 [00:16<00:00,  1.55it/s]


In [67]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How does winning impact the retention of great...,Winning is crucial for retaining great people ...,[The only way a company in that situation can ...,Companies that are winning never have a retent...,1.0,0.927039,0.000000,0.805556,0.688864
1,How does the constant change in the world make...,The constant change in the world makes it diff...,"[tunity, someone else generally will and it wi...",The constant change in the world makes it diff...,1.0,0.964069,1.000000,0.645833,0.333131
2,"What is the concept of ""Task Relevant Maturity...","The concept of ""Task Relevant Maturity"" refers...","[book High Output Management, where he describ...",The concept of 'Task Relevant Maturity' refers...,1.0,0.937940,1.000000,1.000000,0.847315
3,Why is it important to focus on strength rathe...,It is important to focus on strength rather th...,[back — perhaps abstractly — about all of her ...,"When managing executives, it is important to f...",1.0,0.912358,0.333333,1.000000,0.616126
4,What analogy is used to describe the interacti...,The analogy used to describe the interaction b...,"[investors, potential customers, potential par...",The analogy used to describe the interaction b...,0.0,1.000000,0.000000,0.125000,0.232925


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [68]:
results

{'faithfulness': 0.8500, 'answer_relevancy': 0.9336, 'context_recall': 0.6667, 'context_precision': 0.7556, 'answer_correctness': 0.5340}

And see how our advanced retrieval modified our chain!

In [69]:
advanced_retrieval_results

{'faithfulness': 0.8000, 'answer_relevancy': 0.9483, 'context_recall': 0.4667, 'context_precision': 0.7153, 'answer_correctness': 0.5437}

In [70]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.850000,0.800000,-0.050000
1,answer_relevancy,0.933587,0.948281,0.014695
2,context_recall,0.666667,0.466667,-0.200000
3,context_precision,0.755556,0.715278,-0.040278
4,answer_correctness,0.533988,0.543672,0.009684


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [71]:
# Create new OpenAI Embeddigs object with Text Embedding 3 (TE3) Small model for comparision
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [72]:
# Recreate vector store to store embedded vectors using the new TE3 Embedding model
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [73]:
# Cast vector store as retriever 
new_retriever = vector_store.as_retriever()

In [74]:
# We are going to use MultiQueryRetriever with the new Embedding Model
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [75]:
# Combine the advanced retriever with doc chain to create a new retrieval chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [76]:
# Generate responses again through the new retrieval chain
answers = []
contexts = []

# Prepare the necessary lists to align answer and context to input
for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [77]:
# Prepare the dataset by bringing it all together
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions[0:TRUNC_SIZE],
    "answer" : answers[0:TRUNC_SIZE],
    "contexts" : contexts[0:TRUNC_SIZE],
    "ground_truth" : test_groundtruths[0:TRUNC_SIZE]
})

In [78]:
# Finally start the evaluation for the new Embedding Model
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 50/50 [01:53<00:00,  2.27s/it]


In [79]:
# Prit out the relevant results
new_advanced_retrieval_results

{'faithfulness': 0.8750, 'answer_relevancy': 0.9450, 'context_recall': 0.8000, 'context_precision': 0.5696, 'answer_correctness': 0.5133}

In [80]:
# Combine baseline with previous and new results for each comparison
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.850000,0.800000,0.875000,0.075000,0.025000
1,answer_relevancy,0.933587,0.948281,0.944968,-0.003313,0.011382
2,context_recall,0.666667,0.466667,0.800000,0.333333,0.133333
3,context_precision,0.755556,0.715278,0.569571,-0.145706,-0.185984
4,answer_correctness,0.533988,0.543672,0.513261,-0.030412,-0.020727


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

> **Answer:** Based on the limited dataset over which we could run the evaluation due to rate limiting by OpenAI APIs, `TE3` seems marginally better than `ADA`. The main gain it has is on `context_recall`. As goes retrieval, so goes generation. So, this improved retrieval has a positive impact. Though, `context_precision` suffers which could lead to low quality answers. 

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE